Author: Sudipto Ghosh (Roll No. 51)

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
%%writefile data.txt
refrigerator microwave dishwasher freezer juicer expresso
refrigerator microwave dishwasher juicer jewelry
refrigerator microwave dishwasher expresso
microwave juicer jewelry tablet
microwave juicer expresso jewelry tablet
microwave dishwasher freezer juicer expresso
dishwasher freezer juicer expresso
freezer juicer expresso jewelry tablet
dishwasher freezer expresso tablet
dishwasher freezer juicer tablet
tablet speakers jewelry microwave
DSLR printer tablet speakers
freezer DSLR printer
printer tablet jewelry
microwave speakers DSLR
speakers jewelry microwave
speakers jewelry tablet DSLR
refrigerator freezer expresso DSLR
refrigerator microwave DSLR printer
speakers expresso juicer jewelry

Overwriting data.txt


Parse and convert market basket data to a sparse representation.

In [ ]:
db = list()
with open('data.txt') as f:
  for idx, line in enumerate(f.readlines()[:1000]):
    db.append(line.split())

In [ ]:
df = pd.DataFrame({'items': db})
df = df.drop('items', axis=1).join(df['items'].str.join('|').str.get_dummies())

In [ ]:
df

,DSLR,dishwasher,expresso,freezer,jewelry,juicer,microwave,printer,refrigerator,speakers,tablet
0,0,1,1,1,0,1,1,0,1,0,0
1,0,1,0,0,1,1,1,0,1,0,0
2,0,1,1,0,0,0,1,0,1,0,0
3,0,0,0,0,1,1,1,0,0,0,1
4,0,0,1,0,1,1,1,0,0,0,1
5,0,1,1,1,0,1,1,0,0,0,0
6,0,1,1,1,0,1,0,0,0,0,0
7,0,0,1,1,1,1,0,0,0,0,1
8,0,1,1,1,0,0,0,0,0,0,1
9,0,1,0,1,0,1,0,0,0,0,1


Use Apriori principle to construct frequent itemsets that satisfy the `min_sup` threshold.

The code implements the Apriori algorithm for finding frequent itemsets in a transactional dataset. The code takes as input a Pandas dataframe `df`, the minimum support threshold `min_sup`, and the maximum length of itemsets to be considered `max_len`.

Here's a brief overview of what the code does:

* The function `support` calculates the support of each itemset, which is the number of occurrences of each itemset divided by the number of rows in the input data.

* The function `generate_new_combinations` is a generator function that produces combinations of itemsets.

* The main function `apriori_gen` uses the above two functions to implement the Apriori algorithm. The algorithm starts by finding the frequent 1-itemset and storing their supports and indices in dictionaries `support_dict` and `itemset_dict`.

* The algorithm then iterates over the length of the itemsets, using the previous frequent itemsets to generate new combinations, and checking if they meet the minimum support threshold.

* The algorithm continues this process until there are no more frequent itemsets to be found, or until the maximum length of itemsets has been reached.

* The final result is stored in a Pandas dataframe with two columns, `itemsets` and `support`, and returned by the function. The `itemsets` column contains sets of items, and the `support` column contains their corresponding supports.

In [ ]:
def support(x, n):
  """
  calculate support of each itemset as #occurences / #rows
  """
  return np.array(np.sum(x, axis=0) / n).reshape(-1)

def generate_new_combinations(old_combinations):
  """
  generator function that produces combinations of itemsets
  """
  prev_items = np.unique(old_combinations.flatten())
  for old_combination in old_combinations:
      max_combination = old_combination[-1]
      valid_items = prev_items[prev_items > max_combination]
      for item in valid_items:
          yield from old_combination
          yield item

def apriori_gen(df, min_sup, max_len = float("inf")):
  X = df.values

  sup = support(X, X.shape[0])
  col_idx = np.arange(X.shape[1])

  support_dict = {1: sup[sup >= min_sup]}
  itemset_dict = {1: col_idx[sup >= min_sup].reshape(-1, 1)}

  rows_count = float(X.shape[0])
  all_ones = np.ones((int(rows_count), 1))

  max_itemset = 1
  while max_itemset and max_itemset < max_len:
    next_max_itemset = max_itemset + 1

    combin = generate_new_combinations(itemset_dict[max_itemset])
    combin = np.fromiter(combin, dtype=int)
    combin = combin.reshape(-1, next_max_itemset)

    if combin.size == 0:
        break

    _bools = X[:, combin[:, 0]] == all_ones

    for n in range(1, combin.shape[1]):
        _bools = _bools & (X[:, combin[:, n]] == all_ones)
    else:
        _bools = np.all(X[:, combin], axis=2)

    sup = support(np.array(_bools), rows_count)

    if any((sup >= min_sup).reshape(-1)):
        itemset_dict[next_max_itemset] = np.array(combin[(sup >= min_sup).reshape(-1)])
        support_dict[next_max_itemset] = np.array(sup[(sup >= min_sup).reshape(-1)])
        max_itemset = next_max_itemset
    else:
        break

    all_res = []
    for k in sorted(itemset_dict):
        sup = pd.Series(support_dict[k])
        itemsets = pd.Series([set(i) for i in itemset_dict[k]])

        res = pd.concat((itemsets, sup), axis=1)
        all_res.append(res)

    res_df = pd.concat(all_res)
    res_df.columns = ['itemsets', 'support']

    mapping = {idx: item for idx, item in enumerate(df.columns)}

    res_df['itemsets'] = res_df['itemsets'].apply(
        lambda x: set([mapping[i] for i in x])
    )
    res_df = res_df.reset_index(drop=True)

    return res_df

In [ ]:
frequent_itemsets = apriori_gen(df, .25, 5)
print(frequent_itemsets)

                  itemsets  support
0                   {DSLR}     0.30
1             {dishwasher}     0.35
2               {expresso}     0.45
3                {freezer}     0.40
4                {jewelry}     0.45
5                 {juicer}     0.45
6              {microwave}     0.50
7           {refrigerator}     0.25
8               {speakers}     0.30
9                 {tablet}     0.45
10  {expresso, dishwasher}     0.25
11   {freezer, dishwasher}     0.25
12    {juicer, dishwasher}     0.25
13     {freezer, expresso}     0.30
14      {juicer, expresso}     0.30
15       {freezer, juicer}     0.25
16       {jewelry, juicer}     0.25
17    {microwave, jewelry}     0.25
18       {jewelry, tablet}     0.30
19     {microwave, juicer}     0.25


Generate and print strong rules that satisfy the `min_conf` threshold from the given frequent itemsets and their supports.

The code generates association rules from a given pandas dataframe "df". The dataframe is expected to contain two columns "itemsets" and "support". The function returns a dataframe that consists of all the rules that satisfy the minimum confidence threshold of 0.8 (can be changed by passing a different value for the min_conf argument).

The itemsets in the "itemsets" column of the input dataframe are first converted to frozensets and used as keys in a dict to map the corresponding support value. The function then iterates over all frequent itemsets in the dict, and calculates the combinations of antecedent and consequent itemsets. Confidence is calculated as the ratio of the support of the rule to the support of the antecedent. If a rule's confidence satisfies the minimum threshold, it is recorded in the output dataframe along with its support and confidence. The resulting dataframe is sorted by confidence in descending order and returned.

In [ ]:
from itertools import combinations

def gen_rules(df, min_conf=0.8):
  # extract values from the pandas df
  itemsets = df["itemsets"].values
  supports = df["support"].values

  # define a vectorized lambda that converts each itemset to a frozenset
  frozenset_vect = np.vectorize(lambda x: frozenset(x))

  # construct mapping from itemset to support - similar to df but use frozenset
  frequent_items_dict = dict(zip(frozenset_vect(itemsets), supports))

  # initialise lists for storing rule itemsets and supports
  rule_antecedents = []
  rule_consequents = []
  rule_supports = []

  # iterate over all frequent itemsets
  for k in frequent_items_dict.keys():
      # extract its support from the dict
      sXY = frequent_items_dict[k]
      # construct combinations of itemsets of various lengths
      # from the given frequenrt itemsets
      for idx in range(len(k) - 1, 0, -1):
          for c in combinations(k, r=idx):
              X = frozenset(c)
              Y = k.difference(X)

              # extract individual support for antecedent and consequent
              sX = frequent_items_dict[X]
              sY = frequent_items_dict[Y]

              # calculate confidence from the given support counts
              score = sXY / sX

              # record strong rules that satisfy the min_conf threshold
              if score >= min_conf:
                  rule_antecedents.append(X)
                  rule_consequents.append(Y)
                  rule_supports.append([sXY, sX, sY])

  if not rule_supports:
      # return an empty df if there is no rule that satisfies min_conf threshold
      return pd.DataFrame(columns=["X", "Y", "confidence"])
  else:
      # extract supports for all rules into a numpy array
      rule_supports = np.array(rule_supports).T.astype(float)

      # construct result df and add all rules
      res_df = pd.DataFrame(
          data=zip(rule_antecedents, rule_consequents),
          columns=["X", "Y"],
      )

      # extract series containing support of rule components
      sXY = rule_supports[0]
      sX = rule_supports[1]
      sY = rule_supports[2]

      # construct a nicer string representation for rules
      res_df.X = res_df.X.map(lambda x: set(x))
      res_df.Y = res_df.Y.map(lambda y: set(y))
      res_df['rule'] = res_df.X.astype(str) + ' => ' + res_df.Y.astype(str)
      res_df.drop(['X', 'Y'], axis=1, inplace=True)

      # calculate and record conf for all selected rules
      res_df['support'] = sXY
      res_df['confidence'] = sXY / sX

      # return rules and conf sorted by decreasing conf values
      return res_df.sort_values(['confidence'], ascending=False)\
                   .reset_index(drop=True)

In [ ]:
rules_df = gen_rules(frequent_itemsets, 0.6)
print(rules_df)

                              rule  support  confidence
0      {'freezer'} => {'expresso'}     0.30    0.750000
1   {'dishwasher'} => {'expresso'}     0.25    0.714286
2    {'dishwasher'} => {'freezer'}     0.25    0.714286
3     {'dishwasher'} => {'juicer'}     0.25    0.714286
4      {'expresso'} => {'freezer'}     0.30    0.666667
5       {'juicer'} => {'expresso'}     0.30    0.666667
6       {'expresso'} => {'juicer'}     0.30    0.666667
7        {'jewelry'} => {'tablet'}     0.30    0.666667
8        {'tablet'} => {'jewelry'}     0.30    0.666667
9    {'freezer'} => {'dishwasher'}     0.25    0.625000
10       {'freezer'} => {'juicer'}     0.25    0.625000
